<a href="https://colab.research.google.com/github/AjitKhanal8/AjitKhanal8/blob/main/docs/notebooks/00_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/gee-community/geemap/blob/master/docs/notebooks/00_geemap_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Using geemap with Google Colab

## Introduction

This notebook demonstrates how to use [geemap](https://geemap.org) with Google Colab. To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial).

## Install geemap

The geemap package is pre-installed in Google Colab and is updated to the latest minor or major release every few weeks. If you would like to install the latest version, you can uncomment and run the following command:

In [ ]:
# %pip install -U geemap

## Import libraries

Import the Earth Engine Python API.

In [2]:
import ee

## EE Authentication

Running the following cell will start the Earth Engine authentication. Follow the instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication) to authenticate Earth Engine.

In [3]:
ee.Authenticate()

In [4]:
ee.Initialize(project="gis-eof")

## Core features

You can import the geemap package using the following conventions:

- To import only the core features of geemap, use: `import geemap.core as geemap`
- To import all the features of geemap, use: `import geemap`

Let's import the geemap package using the second option:

In [91]:
import geemap

In [103]:
# Create interactive map centered on the region
Map = geemap.Map(center=[45.0, -95.0], zoom=6)

# Define the states of interest with their FIPS codes
states_info = {
    'Minnesota': '27',
    'South Dakota': '46',
    'North Dakota': '38',
    'Wisconsin': '55'
}

In [104]:
# Load US states for reference
states = ee.FeatureCollection('TIGER/2018/States')
target_states_fc = states.filter(ee.Filter.inList('STATEFP', list(states_info.values())))

# Add states outline to map
Map.addLayer(target_states_fc, {'color': 'black', 'width': 2}, 'Target States Outline')

# Display map after adding states
display(Map)

Map(center=[45.0, -95.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [94]:
# Load US counties dataset
counties = ee.FeatureCollection('TIGER/2018/Counties')
# Filter counties for our target states
target_counties = counties.filter(ee.Filter.inList('STATEFP', list(states_info.values())))

In [105]:
# Add counties to map
Map.addLayer(target_counties, {'color': 'gray', 'width': 1}, 'All Counties')


# Display map after adding counties
display(Map)

Map(bottom=6194.0, center=[45.0, -95.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [108]:
# Load land cover data (NLCD 2021)
nlcd = ee.Image('USGS/NLCD_RELEASES/2021_REL/NLCD/2021').select('landcover')

# Create farmland mask (classes 81-82 are cultivated crops)
farmland = nlcd.eq(81).Or(nlcd.eq(82))

# Load water data
water_nlcd = nlcd.eq(11)  # Open Water from NLCD

In [109]:
# Load JRC water data
jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').select('occurrence')
jrc_water_mask = jrc_water.gt(50)  # Areas with >50% water occurrence

# Combine water sources
water_combined = water_nlcd.Or(jrc_water_mask)

In [110]:
# Add reference layers
farmland_vis = {'min': 0, 'max': 1, 'palette': ['yellow']}
Map.addLayer(farmland.updateMask(farmland), farmland_vis, 'Farmland', False)

water_vis = {'min': 0, 'max': 1, 'palette': ['blue']}
Map.addLayer(water_combined.updateMask(water_combined), water_vis, 'Water Bodies', False)


In [111]:
display(Map)

Map(bottom=3102.0, center=[49.32512199104003, -96.63574218750001], controls=(WidgetControl(options=['position'…

In [112]:
def analyze_county_borders(county):
    """Analyze if a county has farmland bordered by water"""

    # Get county geometry
    geom = county.geometry()

    # Clip data to county
    county_farmland = farmland.clip(geom)
    county_water = water_combined.clip(geom)

    # Create a buffer around water bodies (50m)
    water_buffered = county_water.focal_max(
        radius=50,
        kernelType='square',
        units='meters'
    )

    # Find farmland that intersects with buffered water
    farmland_near_water = county_farmland.And(water_buffered)

    # Calculate pixel counts
    farmland_pixels = county_farmland.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=30,
        maxPixels=1e9,
        bestEffort=True
    ).get('landcover')

    border_pixels = farmland_near_water.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=30,
        maxPixels=1e9,
        bestEffort=True
    ).get('landcover')

    # Convert to numbers and handle null values
    farmland_count = ee.Number(farmland_pixels).max(0)
    border_count = ee.Number(border_pixels).max(0)

    # Determine if county has significant farmland-water borders
    has_border = ee.Algorithms.If(
        border_count.gt(10).And(
            border_count.divide(farmland_count.max(1)).gt(0.005)
        ),
        1,
        0
    )

    # Calculate ratio
    ratio = border_count.divide(farmland_count.max(1))

    return county.set({
        'farmland_pixels': farmland_count,
        'border_pixels': border_count,
        'border_ratio': ratio,
        'has_farmland_water_border': has_border
    })

In [113]:
# Apply the analysis to all counties
analyzed_counties = target_counties.map(analyze_county_borders)


In [114]:
counties_with_borders = analyzed_counties.filter(
    ee.Filter.eq('has_farmland_water_border', 1)
)
counties_without_borders = analyzed_counties.filter(
    ee.Filter.eq('has_farmland_water_border', 0)
)

In [115]:
counties_with_borders_styled = counties_with_borders.style(
    color='red',
    width=3,
    fillColor='red'
)

# Style counties without borders (BLUE)
counties_without_borders_styled = counties_without_borders.style(
    color='green',
    width=2,
    fillColor='green'
)

In [116]:
Map.addLayer(
    counties_with_borders_styled,
    {'opacity': 0.7},
    'Counties WITH Farmland-Water Borders'
)

Map.addLayer(
    counties_without_borders_styled,
    {'opacity': 0.3},
    'Counties WITHOUT Farmland-Water Borders'
)

In [117]:
# Display final map
display(Map)

Map(bottom=1869.0, center=[38.75408327579141, -99.228515625], controls=(WidgetControl(options=['position', 'tr…

In [118]:
def get_summary_stats():
    """Get summary statistics for each state"""

    results = {}

    for state_name, fips_code in states_info.items():
        state_counties = analyzed_counties.filter(ee.Filter.eq('STATEFP', fips_code))

        with_borders_state = state_counties.filter(ee.Filter.eq('has_farmland_water_border', 1))
        without_borders_state = state_counties.filter(ee.Filter.eq('has_farmland_water_border', 0))

        try:
            total_count = state_counties.size().getInfo()
            with_count = with_borders_state.size().getInfo()
            without_count = without_borders_state.size().getInfo()

            results[state_name] = {
                'total': total_count,
                'with_borders': with_count,
                'without_borders': without_count,
                'percentage_with_borders': round((with_count/total_count)*100, 1) if total_count > 0 else 0
            }

        except Exception as e:
            print(f"Error getting stats for {state_name}: {e}")
            results[state_name] = {'error': str(e)}

    return results

# Get and display summary
summary = get_summary_stats()

In [119]:
print("\n" + "="*60)
print("FARMLAND-WATER BORDER ANALYSIS SUMMARY")
print("="*60)

for state, stats in summary.items():
    if 'error' not in stats:
        print(f"\n{state}:")
        print(f"  Total counties: {stats['total']}")
        print(f"  Counties with farmland-water borders: {stats['with_borders']}")
        print(f"  Counties without borders: {stats['without_borders']}")
        print(f"  Percentage with borders: {stats['percentage_with_borders']}%")

print("\n" + "="*60)
print("Map Legend:")
print("🔴 RED = Counties with farmland-water borders (value = 1)")
print("🔵 BLUE = Counties without farmland-water borders (value = 0)")
print("Reference layers (farmland, water) can be toggled on/off")
print("="*60)


FARMLAND-WATER BORDER ANALYSIS SUMMARY

Minnesota:
  Total counties: 87
  Counties with farmland-water borders: 43
  Counties without borders: 44
  Percentage with borders: 49.4%

South Dakota:
  Total counties: 66
  Counties with farmland-water borders: 34
  Counties without borders: 32
  Percentage with borders: 51.5%

North Dakota:
  Total counties: 53
  Counties with farmland-water borders: 32
  Counties without borders: 21
  Percentage with borders: 60.4%

Wisconsin:
  Total counties: 72
  Counties with farmland-water borders: 35
  Counties without borders: 37
  Percentage with borders: 48.6%

Map Legend:
🔴 RED = Counties with farmland-water borders (value = 1)
🔵 BLUE = Counties without farmland-water borders (value = 0)
Reference layers (farmland, water) can be toggled on/off


In [120]:
# Function to export results
def export_to_drive():
    """Export results to Google Drive"""
    task = ee.batch.Export.table.toDrive(
        collection=analyzed_counties,
        description='farmland_water_border_counties',
        folder='GEE_Exports',
        fileFormat='CSV',
        selectors=[
            'STATEFP', 'COUNTYFP', 'NAME', 'STUSPS',
            'has_farmland_water_border', 'border_ratio',
            'farmland_pixels', 'border_pixels'
        ]
    )
    task.start()

In [121]:
Map

Map(bottom=3295.0, center=[43.48481212891603, -97.25097656250001], controls=(WidgetControl(options=['position'…